<a href="https://colab.research.google.com/github/Robinkim0125/DataAnalysis/blob/main/WebCrawling(naver_news).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 뉴스 크롤링 실습

오늘 실습 과정에 대한 간략한 설명!

1. 네이버 뉴스 페이지에서, 각각의 뉴스가 가진 URL을 리스트 형태로 만들어서 저장해놓는다.
2. 각각의 뉴스가 가진 URL 에 접근하면 제목과 본문 내용이 표시가 될텐데, 그 제목과 본문 내용을 newspaper3k 패키지를 이용해서 크롤링을 진행한다.
3. 크롤링을 할 때마다 크롤링 되어온 내용을 데이터프레임에 계속 이어붙인다.
4. 크롤링이 끝났다면, 정보가 저장되어있는 데이터프레임을 csv 파일로 저장한다.

크롤러를 만들기 전에 우선 필요한 도구들을 임포트합니다.

In [ ]:
!pip install newspaper3k

In [ ]:
import requests
import pandas as pd
from newspaper import Article
from bs4 import BeautifulSoup

변수를 이용해서 URL을 완성하는 연습을 해봅시다. 앞으로 자동으로 네이버 뉴스 URL에 접속하게 될텐데, 이 중 카테고리 번호, 날짜, 페이지 번호의 경우만 바꾸면 해당 카테고리, 날짜, 페이지의 URL에 접근할 수 있습니다.

페이지번호를 의미하는 변수로 page_num을 선언하고, 카테고리를 의미하는 변수로 code를 선언하고, 날짜를 의미하는 변수로 date를 선언합시다. 숫자로 변수를 선언하면 정수형 자료형이 되지만, URL이라는 것은 실제로는 문자열입니다.

그래서 조합하기 전에 정수형 변수에 str(변수명)을 해주게 되면, 해당 변수는 문자열 자료형으로 변경되므로 다른 문자열들과 조합이 가능하게 됩니다. 조합 방법은 더하기 기호인 +로 해주시면 됩니다. 자, 그러면 **2020년 5월 6일**에 **경제(101번 카테고리)** 뉴스란의 **1페이지** URL을 만들어볼까요?

In [ ]:
page_num = 1
code = 101
date = 20200506

# 문자열은 더하면 이어붙이기가 된다
# 문자열과 숫자는 더할 수 없다 -> 숫자를 문자열로 바꿔주는 작업을 해야한다
url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(page_num) 
print(url)

https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101&date=20200506&page=1


완성된 URL 클릭하여 실제 네이버 뉴스 페이지로 이동해봅시다.  

변수를 이용해서 URL을 만드는 법을 이해했다면, 이제 해당 URL에 있는 HTML 소스 코드를 가져오는 방법을 이해해야 합니다. 이는 requests라는 패키지의 get이라는 모듈을 통해서 가능합니다.

requests라는 패키지에 있는 get이라는 모듈(함수)에 url을 입력으로 하고, 이를 변수에 저장한 후에 변수.content를 하면 해당 URL의 HTML 코드를 받아올 수 있습니다. 앞서 만든 URL로 테스트를 해봅시다.

In [ ]:
news = requests.get(url)
news.content

어라? HTML 코드가 나오기는 하는데, 우리가 예상한 HTML 코드가 아니라 굉장히 짧은 HTML 코드가 출력됩니다. 그리고 중간에 'Access Denied'와 'Request is denied' 라는 문장이 적혀져 있는데요. 네. 저희는 접근을 차단당했습니다. 네이버가 보기에 방금 접근이 크롤링 행위를 하는 것으로 추정되어서 네이버가 저희를 차단한 것인데요. 이런 경우에는 일종의 편법을 사용해서 사용자인척 해야합니다.

이에 대한 자세한 내용은 아래의 링크를 통해 확인할 수 있습니다.

https://hogni.tistory.com/64

위의 링크에서 설명하고 있듯이, 유저 에이전트를 인자로 넣어서 get 함수를 사용하면 위와 같은 차단을 뚫고 크롤링을 할 수 있습니다. 한 번 다시 해볼까요? 아래의 headers는 강사의 브라우저에서 추출한 값으로 여러분들은 여러분들의 에이전트 정보를 추출해서 사용하셔도 됩니다!

In [ ]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}

In [ ]:
news = requests.get(url, headers=headers)
news.content

이번에는 정상적으로 해당 페이지의 HTML 코드가 추출된 것을 확인할 수 있습니다! 이제 이를 이용해서 원하는 날짜의, 원하는 카테고리의, 원하는 페이지에 접속해서 해당 페이지에 있는 뉴스 기사들의 URL을 전부 뽑아내는 함수를 만들어봅시다.

다음은 BeautifulSoup를 이용해서 원하는 페이지 수와, 카테고리 번호, 날짜를 입력하면 해당 URL에 접속하여 뉴스 url 리스트를 리턴하는 함수입니다.

In [ ]:
# 각 기사의 URL 주소를 가져와서 리스트에다가 담아주는 함수
# 크롤링할 페이지 수, 카테고리, 날짜
def make_urllist(page_num, code, date): 
  # 각각 기사들이 가진 url을 저장하기 위한 빈 리스트를 하나 만들어놓는다.
  urllist= []

  # 1 ~ page_num까지 정해진 페이지만큼 반복.
  for i in range(1, page_num + 1):

		# 함수의 입력으로 된 변수들로 주소를 조합
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   

		# requets 패키지의 모듈(함수)을 호출
		# import 패키지이름 으로 '임포트'했다면 '패키지이름.모듈이름' 으로 모듈을 호출합니다.
    news = requests.get(url, headers=headers)
    news.content

    # BeautifulSoup 모듈을 사용하여 HTML 페이지를 분석
    soup = BeautifulSoup(news.content, 'html.parser')

    # 해당 페이지에 있는 각각의 뉴스가 news_list 라는 리스트로 저장됩니다!
    news_list = soup.select('.type06_headline li dl')
    news_list.extend(soup.select('.type06 li dl'))

    # 뉴스 리스트에 있는 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져온다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

어디 한 번 함수를 실행해볼까요? 저는 2020년 5월 6일, 경제 기사(코드로 101번)를 2페이지까지 탐색해서 URL 리스트를 받아오겠습니다. 한 페이지당 뉴스 기사가 20개가 있으니까, 2페이지까지 탐색하면 총 40개의 URL 리스트를 받아와야 합니다.

In [ ]:
url_list = make_urllist(2, 101, 20200506)
print('뉴스 기사의 개수 :',len(url_list))

뉴스 기사의 개수 : 40


총 40개의 URL이 존재하는 것을 확인할 수 있습니다. 5개만 출력해봅시다.

In [ ]:
url_list[:5]

['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451723',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451721',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451718',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0009849190',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451717']

뉴스 URL들이 잘 저장되어져 있는 것을 확인할 수 있습니다. 뉴스 URL 리스트를 받아올 수 있다면, 이제 newspaper3k를 이용해서 뉴스 기사들을 파이썬을 저장할 수 있겠군요!

앞으로 결과를 확인할 때 코드로부터 바로 어떤 카테고리인지 확인하기 쉽도록 code를 key, 실제 카테고리를 value로 가지는 dictionary를 만들어두겠습니다. 이번 실습에 사용할 카테고리들에 대해서만 생성했습니다.

참고 : https://wikidocs.net/16

In [ ]:
# 파이썬의 dictionarys는 선언 할 때는 'key' : 'value'의 형태로 선언을 합니다.
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

In [ ]:
# 선언 후에는 key 값을 넣으면 맵핑되어져 있는 value 값이 리턴됩니다.
idx2word['101']

'경제'

In [ ]:
idx2word['105']

'IT/과학'

이제 idx2word라는 딕셔너리를 통해서 이 코드 번호가 무슨 카테고리였지? 하고 생각이 안 날때마다 idx2word에 입력을 넣어서 확인할 수 있습니다.

다음은 newspaper3k를 통해서 만들어진 함수로 url 리스트와 해당 url이 어떤 카테고리인지 코드를 알려주면 이를 통해 데이터프레임을 생성하는 함수입니다.

In [ ]:
# url과 카테고리를 알려주면 url로부터 본문을 추출하여 데이터프레임을 생성하는 함수.
def make_data(urllist, code):
  # 본문을 저장하기 위한 빈 리스트를 만들어놓는다.
  text_list = []
  
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  df = pd.DataFrame({'news': text_list})
  df['code'] = idx2word[str(code)] 
  return df

앞서 저장해둔 경제 카테고리(코드 101)의 40개의 url 리스트로부터 데이터프레임을 생성합니다.

In [ ]:
data = make_data(url_list, 101)
data[:10] # 상위 10개 출력

,news,code
0,고려은단이 5월을 맞아 응원 메시지를 공유하는 ‘5월 5글자로 응원 부탁해!’ 이벤...,경제
1,코리아나화장품의 민감성 피부를 위한 저자극 스킨케어 브랜드 '프리엔제'가 마르고 건...,경제
2,서울장수주식회사가 부드럽고 달콤한 맛으로 인기를 모으고 있는 생막걸리 ‘인생막걸리’...,경제
3,[서울=뉴시스] 오동현 기자 = 모바일 게임 기업 컴투스는 3D 모바일 야구 게임 ...,경제
4,대원제약이 2020년 상반기 신입과 경력 정기 공채를 실시합니다.정기 공채 모집분야...,경제
5,"[AFP=연합뉴스] [AFP=연합뉴스]\n\n""요즘은 잔인한 날""…리프트도 앞서 9...",경제
6,이재용 삼성전자 부회장이 6일 삼성전자 서울 서초사옥에서 대국민 사과 회견을 하기 ...,경제
7,JW중외제약이 A형 혈우병 예방요법제 ‘헴리브라피하주사를 출시하고 본격적인 마케팅 ...,경제
8,"옵티팜과 휴벳바이오가 공동 개발중인 백신 후보 물질에 대해 마우스, 기니피그, 미니...",경제
9,[한국경제TV 신동호 기자]\n\n전남 나주시와 충북 청주시가 방사광 가속기 구축사...,경제


이번에는 하나의 카테고리만이 아니라 다른 카테고리의 뉴스들에 대해서도 수집을 하고자 합니다. 가령, 특정 날짜의 사회, 생활/문화, IT/과학의 뉴스들을 수집해본다고 해볼게요. 수집을 원하는 카테고리 코드들을 저장한 리스트를 만들어둡니다.

In [ ]:
code_list = [102, 103, 105]

이 코드 리스트, 그리고 날짜, 페이지 수를 입력으로 받는 make_total_data라는 함수를 만듭니다. 이 함수는 내부적으로 앞서 만든 make_urllist 함수와 make_data 함수를 호출하도록 합니다.

In [ ]:
# 원하는 날짜, 원하는 페이지 수, 수집을 원하는 카테고리를 입력.
def make_total_data(page_num, code_list, date):
  # 아무것도 들어있지 않은 데이터 프레임을 만든다 
  df = None

  # 각 카테고리에 대해서 아래의 코드를 수행.
  for code in code_list:

    # make_urllist 함수 : 페이지 수, 카테고리, 날짜를 입력하면 url을 추출하는 함수.
    url_list = make_urllist(page_num, code, date)

    # make_data 함수: url과 카테고리를 알려주면 데이터프레임을 만들어주는 함수
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None: # df가 채워져있나요?
      # for문 내에서 데이터프레임이 새로 만들어질 때마다
      # 이전 데이터프레임과 지속적으로 결합
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

5월 6일 기사에 대해서 2페이지까지, 102번, 103번, 105번 코드를 가지는 카테고리의 뉴스들을 수집해서 데이터프레임에 저장해보겠습니다.

In [ ]:
df = make_total_data(2, code_list, 20200506)

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


1페이지당 20개의 뉴스 기사가 존재하므로, 총 120개의 뉴스기사가 수집되어야 합니다.  
2(페이지 수) x 20(페이지당 url) x 3(카테고리 수) = 120

In [ ]:
print('뉴스 기사의 개수 :',len(df)) # len은 데이터 길이를 출력하는 함수

뉴스 기사의 개수 : 120


120개의 뉴스 기사가 수집되었네요. 임의로 10개의 샘플을 선택하여 출력해보겠습니다.

(임의 출력이므로 아래 결과는 사람마다 다를 수 있습니다.)

In [ ]:
df.sample(10)

,news,code
1,"[AFP=연합뉴스] [AFP=연합뉴스]\n\n""요즘은 잔인한 날""…리프트도 앞서 9...",IT/과학
34,김성태 국회 과학기술정보방송통신위원회 정보통신방송법안심사소위원회 위원장이 6일 서울...,IT/과학
32,라임자산운용의 전주로 지목된 김봉현 스타모빌리티 회장. 연합뉴스\n\n경찰이 ‘라임...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
32,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,IT/과학
9,기사 섹션 분류 안내\n\n기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다...,생활/문화
31,6일 서울 여의도 국회의사당 앞에서 양육비해결총연합회 관계자들이 양육비 이행확보 및...,사회
11,코로나19로 텅빈 베로나 아레나와 브라광장매년 여름 오페라 축제가 열리는 베로나 아...,생활/문화
14,지난달 23일 오후 강원 춘천시청 대회의실에서 열린 제2회 강원 미래 과학 포럼에서...,IT/과학
35,(청주=연합뉴스) 전창해 기자 = '꿈의 빛'으로 불리는 차세대 다목적 방사광가속기...,IT/과학


10개의 샘플이 출력해보았는데, 3개의 카테고리가 전부 존재하는 것을 확인할 수 있습니다. df.sample(10)을 여러번 호출하면서 달라지는 결과를 확인해보세요. 수집한 데이터의 샘플들을 랜덤으로 여러번 출력해보는 것은 데이터를 파악하는데 꽤 큰 도움이 됩니다.

**이제 뉴스 크롤러가 완성되었습니다!** 이번에는 좀 더 많은 데이터를 수집해볼까요? 이번에는 3개의 카테고리에 대해서 **총 10개의 페이지**에 대해서 크롤링하겠습니다. 

크롤링해서 수집되는 전체 뉴스의 양은 네이버 측의 상황에 따라서 코드를 실행하는 날에 따라 다소 다를 수 있음이 확인되었습니다. 향후 실습에서 제가 최종 수집한 뉴스기사와 여러분들이 동일한 코드를 실행해서 얻은 뉴스 기사의 개수가 다소 다르더라도 신경쓰지 않으셔도 됩니다.

In [ ]:
df = make_total_data(10, code_list, 20200506)

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


위 코드는 최소 수십분의 시간이 걸립니다. 혹시나 빠르게 진행하고 싶으시다면 첫번째 인자에 10이 아니라 1을 넣으셔도 괜찮습니다.

In [ ]:
df

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회
...,...,...
195,(지디넷코리아=이도원 기자)스마일게이트알피지(대표 지원길)는 블록버스터 핵앤슬래시 ...,IT/과학
196,모두의셔틀 로고 (모두의셔틀 제공) © 뉴스1 모두의셔틀 로고 (모두의셔틀 제공) ...,IT/과학
197,휴온스의 신종 코로나바이러스 감염증(코로나19) 관련 방역용품이 미국 수출길에 오른...,IT/과학
198,http://news.kbs.co.kr/news/list.do?icd=19588\n...,IT/과학


In [ ]:
df.to_csv('news_data.csv', index=False)

In [ ]:
df = pd.read_table('news_data.csv', sep=',')

In [ ]:
df

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회
...,...,...
595,(지디넷코리아=이도원 기자)스마일게이트알피지(대표 지원길)는 블록버스터 핵앤슬래시 ...,IT/과학
596,모두의셔틀 로고 (모두의셔틀 제공) © 뉴스1 모두의셔틀 로고 (모두의셔틀 제공) ...,IT/과학
597,휴온스의 신종 코로나바이러스 감염증(코로나19) 관련 방역용품이 미국 수출길에 오른...,IT/과학
598,http://news.kbs.co.kr/news/list.do?icd=19588\n...,IT/과학
